{
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "{\n",
                "    \"cells\": [\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"# Temperature sensitivity of CO2 emissions in layered cranberry soils\\n\",\n",
                "                \"*Wilfried Dossou-Yovo, Serge Étienne Parent,Noura Ziadi, Bertrand Normand, and Léon Étienne parent*\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"print(Sys.Date())\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"## 1. Objective\\n\",\n",
                "                \"\\n\",\n",
                "                \"This notebook generate the result of CO2 data analysis. Data set contains a collection of soil caracteristics, measured co2 emission collected from incubation study. Soil samples was collected from two cranberry fied stand of eastern canada. Incubation study was carried out at Agriculture and Agri-food Canada(sainte-foy, quebec,qc) from February to Mai 2019. The aim of this study was to measure CO2 emission rates in cranberry soils of Eastern Canada as related to soil temperature and depth\\n\",\n",
                "                \"\\n\",\n",
                "                \"## 2. Statistical questions\\n\",\n",
                "                \"\\n\",\n",
                "                \"In addition to data exploration, this notebook will answer the following statistical questions.\\n\",\n",
                "                \" \\n\",\n",
                "                \"1. What is the influence of soil depth and temperature on CO2 emission?\\n\",\n",
                "                \"2. Can Arrhenius equation and Q10 be useful to describe temperature sensitivity of carbon decomposition across layers? \\n\",\n",
                "                \"\\n\",\n",
                "                \"## 3. Packages\\n\",\n",
                "                \"\\n\",\n",
                "                \"We need package [tidyverse](https://www.tidyverse.org/) which loads a set of packages for easy data manipulation(Ex: [dplyr](https://www.rdocumentation.org/packages/dplyr/versions/0.7.8)) and visualization (ex: [ggplot2](https://www.rdocumentation.org/packages/ggplot2/versions/3.2.1)). We also use [ggpubr](https://www.rdocumentation.org/packages/ggpubr/versions/0.1.1) to customise publication ready plot, [ggpmisc](https://www.rdocumentation.org/packages/ggpmisc/versions/0.2.17) and [grid](https://www.rdocumentation.org/packages/grid/versions/3.6.2) are useful packages as extensions to [ggplot2](https://www.rdocumentation.org/packages/ggplot2/versions/3.2.1).\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"#install.packages(c(\\\"ggpubr\\\", \\\"ggpmisc\\\", \\\"tidyverse\\\", \\\"tidymodels\\\", \\\"plyr\\\", \\\"plotly\\\"))\\n\",\n",
                "                \"library(plyr)\\n\",\n",
                "                \"library(tidyverse)\\n\",\n",
                "                \"library(tidymodels)\\n\",\n",
                "                \"library(ggpubr)\\n\",\n",
                "                \"library(ggpmisc)\\n\",\n",
                "                \"library(grid)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"### 4. Import data\\n\",\n",
                "                \"We load two data `data_pot` and `data_co2` involved in our anylisis. `data_pot` contained details about sites sampling, soil sampling(soil depth, weight, water content and bulk density), laboratory incubation temperature while `data_co2` contained details about laboratory incubation time, co2 emission and jar masson details.\\n\",\n",
                "                \"`data_co2` was combined with `data_pot` with `left_join` function\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"data_pots <- read_csv2('data/pots.csv')\\n\",\n",
                "                \"data_co2 <- read_csv('data/co2.csv')\\n\",\n",
                "                \"data_co2 <- data_co2 %>%\\n\",\n",
                "                \"left_join(data_pots, by = \\\"ID pot\\\")\\n\",\n",
                "                \"data_pots\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"## 5. Some calculations \\n\",\n",
                "                \"\\n\",\n",
                "                \"Several variables have been added to our data in order to proceed for analysis. The added variables are the following: `Temperature (Kelvin)`, `Molar Volume (L/mol)`, `Headspace Volume (mL)`, `Dry soil weight (g)`, `CO2 emission (ug/h/g)`, `CO2 emission (mg/kg)`, `decomposition rate K`, `lnK`and `1/T(T = Temperature(Kelvin)`\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"container_volume <- 250 # mL\\n\",\n",
                "                \"gas_constant <- 8.31446 # unit\\n\",\n",
                "                \"atm_pressure_n <- 101.325\\n\",\n",
                "                \"\\n\",\n",
                "                \"data_co2 <- data_co2 %>%\\n\",\n",
                "                \"  mutate(`Temperature (K)` = `Temperature (°C)` + 273,\\n\",\n",
                "                \"         `Total carbon (Mg/ha)` = `Bulk density (g/cm3)` * `Carbone(%)` * 10,\\n\",\n",
                "                \"         `Molar Volume (L/mol)` = gas_constant * `Temperature (K)` / atm_pressure_n,\\n\",\n",
                "                \"         `Headspace Volume (mL)` = container_volume - (`Soil weight (g)` / `Bulk density (g/cm3)`), # 250 mL is the volume of the container\\n\",\n",
                "                \"         `Dry soil weight (g)` = `Soil weight (g)` - (`Soil weight (g)` * `Water content (%)` / 100),\\n\",\n",
                "                \"         `CO2 emission (ug/h/g)` = (`Final CO2 (ppm)` - `Initial CO2 (ppm)`) * 0.000001 * 44000000 /\\n\",\n",
                "                \"                                   `Molar Volume (L/mol)` * (`Headspace Volume (mL)` / 1000) * (12 / 44) /\\n\",\n",
                "                \"                                   `Time final (h)` / `Dry soil weight (g)`,\\n\",\n",
                "                \"         `CO2 emission (mg/kg)` = `CO2 emission (ug/h/g)` * 24 * `Time (days)`,\\n\",\n",
                "                \"         `CO2 emission (Mg/ha)` = `CO2 emission (mg/kg)` * `Bulk density (g/cm3)` * 10 * 0.0001,\\n\",\n",
                "                \"         K = log(`Total carbon (Mg/ha)` / (`Total carbon (Mg/ha)` - `CO2 emission (Mg/ha)`)) / `Time (days)`,\\n\",\n",
                "                \"         lnK = log(K),\\n\",\n",
                "                \"         `1/T` = 1 / `Temperature (K)`)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"## 6. Exploratory data analysis\\n\",\n",
                "                \"\\n\",\n",
                "                \"### 6.1 Histogram\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"New.labs <- c(\\\"10°C\\\", \\\"20°C\\\", \\\"30°C\\\") # Change labels \\n\",\n",
                "                \"names(New.labs) <- c(\\\"10\\\", \\\"20\\\", \\\"30\\\")\\n\",\n",
                "                \"\\n\",\n",
                "                \"New.labs_b <-  c(\\\"[0-10 cm]\\\", \\\"[10-20 cm]\\\", \\\"[20-30 cm]\\\") # Change labels\\n\",\n",
                "                \"names(New.labs_b) <- c(\\\"10\\\", \\\"20\\\", \\\"30\\\")\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"library(plotly)\\n\",\n",
                "                \"ggplotly(  \\n\",\n",
                "                \"data_co2 |>\\n\",\n",
                "                \"  ggplot() +\\n\",\n",
                "                \"  geom_histogram(aes(x = log10(`CO2 emission (ug/h/g)`)), bins = 100)\\n\",\n",
                "                \")\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"Data contains some outliers, let remove them\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"data_co2_clean <- data_co2 |> \\n\",\n",
                "                \"  mutate(log_tr = log10(`CO2 emission (ug/h/g)`)) |>\\n\",\n",
                "                \"  filter(log_tr > -3 & log_tr < -0.24) |>\\n\",\n",
                "                \"  drop_na()\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"\\n\",\n",
                "                \"Now data are well distributed\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"ggplotly(  \\n\",\n",
                "                \"data_co2_clean |>\\n\",\n",
                "                \"  ggplot() +\\n\",\n",
                "                \"  geom_histogram(aes(x = log10(`CO2 emission (ug/h/g)`)), bins = 100) \\n\",\n",
                "                \")\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"### 6.2 Correlations\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"data_co2_clean |>\\n\",\n",
                "                \"  select(`Time (days)`, `Depth (cm)`, `Temperature (°C)`, `CO2 emission (ug/h/g)`) |>\\n\",\n",
                "                \"  corrr::correlate() |>\\n\",\n",
                "                \"  corrr::focus(`CO2 emission (ug/h/g)`) |>\\n\",\n",
                "                \"  mutate(term = fct_reorder(term, `CO2 emission (ug/h/g)`)) |>\\n\",\n",
                "                \"  ggplot(aes(x = `CO2 emission (ug/h/g)`, y= term)) +\\n\",\n",
                "                \"  geom_col(width = 0.2) +\\n\",\n",
                "                \"  labs(x = bquote(~CO[2]~ 'emision ('*mu~'g'~ h^-1~g^-1*')')) +\\n\",\n",
                "                \"  theme_bw()\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"### 6.3 Boxplot\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"options(repr.plot.width = 6, repr.plot.height = 7)\\n\",\n",
                "                \"pg <- ggplot(data=data_co2_clean, aes(x = `Time (days)`,y = `CO2 emission (ug/h/g)` )) +\\n\",\n",
                "                \"geom_boxplot(aes(group = factor(`Time (days)`))) + \\n\",\n",
                "                \"facet_grid(`Depth (cm)` ~ `Temperature (°C)`,   scales = \\\"free\\\", \\n\",\n",
                "                \"          labeller = labeller(`Depth (cm)` = New.labs_b, `Temperature (°C)` = New.labs))+ \\n\",\n",
                "                \" labs(x = \\\"Incubation time (days)\\\", y = bquote(~CO[2]~ 'emision ('*mu~'g'~ h^-1~g^-1*')'))\\n\",\n",
                "                \"pg \\n\",\n",
                "                \"ggsave(\\\"figures/Boxplot.png\\\", width = 6, height = 7, dpi = 600)# export plot high resolution\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"## 7.What is the influence of soil depth and temperature on CO2 emission?\\n\",\n",
                "                \"\\n\",\n",
                "                \"### 7.1 Build model: linear regression\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"model_rec <-  data_co2_clean |>\\n\",\n",
                "                \"  recipe(`CO2 emission (ug/h/g)` ~ ., data_co2) |>\\n\",\n",
                "                \"  step_select(`CO2 emission (ug/h/g)`, `Time (days)`, `Depth (cm)`, `Temperature (°C)`) |>\\n\",\n",
                "                \"  step_log(all_outcomes(), base = 10) |>\\n\",\n",
                "                \"  step_normalize(all_numeric(), -all_outcomes() ) |>\\n\",\n",
                "                \"  prep()\\n\",\n",
                "                \"\\n\",\n",
                "                \"data_co2_preprocessed <-  juice(model_rec)\\n\",\n",
                "                \"\\n\",\n",
                "                \"model_spec <- linear_reg() |>\\n\",\n",
                "                \"  set_engine(\\\"lm\\\")\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"\\n\",\n",
                "                \"#### Fit model\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"model_fit <- model_spec |>\\n\",\n",
                "                \"  fit(`CO2 emission (ug/h/g)` ~ ., data_co2_preprocessed)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"\\n\",\n",
                "                \"#### Exploring model results\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"tidy(model_fit)\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"glance(model_fit)\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"\\n\",\n",
                "                \"#### Inspect the model\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"par(mfrow=c(2,2)) # plot all 4 plots in one\\n\",\n",
                "                \"\\n\",\n",
                "                \"plot(model_fit$fit, \\n\",\n",
                "                \"     pch = 16,    \\n\",\n",
                "                \"     col = '#006EA1')\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"\\n\",\n",
                "                \"#### Prediction\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"prediction <-  model_fit |>\\n\",\n",
                "                \"  predict(data_co2_preprocessed)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"\\n\",\n",
                "                \"#### collect Metrics\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"rmse <-  data_co2_preprocessed |>\\n\",\n",
                "                \"  bind_cols(prediction) |>\\n\",\n",
                "                \"  rmse(`CO2 emission (ug/h/g)`, .pred)\\n\",\n",
                "                \"rmse\\n\",\n",
                "                \"rmse <-  round(as.numeric(rmse[1,3]), 2)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"rsq <-  data_co2_preprocessed |>\\n\",\n",
                "                \"  bind_cols(prediction) |>\\n\",\n",
                "                \"  rsq(`CO2 emission (ug/h/g)`, .pred)\\n\",\n",
                "                \"rsq\\n\",\n",
                "                \"rsq <- round(as.numeric(rsq[1,3]), 2)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"data_co2_preprocessed |>\\n\",\n",
                "                \"  bind_cols(prediction) |>\\n\",\n",
                "                \"  ggplot(aes(x = `CO2 emission (ug/h/g)`, y = .pred)) +\\n\",\n",
                "                \"  geom_point() +\\n\",\n",
                "                \"  geom_label(aes(x = -3, y = -0.5),\\n\",\n",
                "                \"              vjust = 1, hjust = 0,\\n\",\n",
                "                \"               label = paste(\\\"R² =\\\",  rsq, \\\"\\\\nRMSE =\\\", rmse)) +\\n\",\n",
                "                \"  geom_abline(color = \\\"red\\\") +\\n\",\n",
                "                \"  labs(x= bquote(\\\"Observed log\\\"~CO[2]~ 'emision ('*mu~'g'~ h^-1~g^-1*')') , y = bquote(\\\"Predicted log\\\"~CO[2]~ 'emision ('*mu~'g'~ h^-1~g^-1*')'))\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"### 7.3 Variable coefficient and confidence intervals\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"options(repr.plot.width = 8, repr.plot.height = 2)\\n\",\n",
                "                \"h <- broom::tidy(model_fit, conf.int = TRUE) |>\\n\",\n",
                "                \"  dplyr::filter(term != \\\"(Intercept)\\\") |>\\n\",\n",
                "                \"  ggplot(aes(estimate, term)) +\\n\",\n",
                "                \"  geom_vline(xintercept = 0, linetype = 2) +\\n\",\n",
                "                \"  geom_point() +\\n\",\n",
                "                \"  geom_errorbarh(aes(xmin = conf.low, xmax = conf.high), height = 0.1,\\n\",\n",
                "                \"                  size=0.5) +\\n\",\n",
                "                \"  labs(x = \\\"Coefficient\\\", y = \\\"\\\") +\\n\",\n",
                "                \"  theme_light()\\n\",\n",
                "                \"h + theme(axis.text = element_text(face = \\\"bold\\\"))\\n\",\n",
                "                \"ggsave(\\\"figures/Linear-model-Co2.png\\\", width = 8, height = 2, dpi = 600)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"### 7.3 Prediction model of CO2 emission in cranberry soils in three-layer positions (0-10 cm, 10-20 cm, 20-30 cm) and at three temperatures (10, 20 and 30oC).\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"New.labs <- c(\\\"10°C\\\", \\\"20°C\\\", \\\"30°C\\\") # Change labels \\n\",\n",
                "                \"names(New.labs) <- c(\\\"10\\\", \\\"20\\\", \\\"30\\\")\\n\",\n",
                "                \"\\n\",\n",
                "                \"New.labs_b <-  c(\\\"[0-10 cm]\\\", \\\"[10-20 cm]\\\", \\\"[20-30 cm]\\\") # Change labels\\n\",\n",
                "                \"names(New.labs_b) <- c(\\\"10\\\", \\\"20\\\", \\\"30\\\")\\n\",\n",
                "                \"\\n\",\n",
                "                \"options(repr.plot.width = 8, repr.plot.height = 6)\\n\",\n",
                "                \"pl <- data_co2_clean |>\\n\",\n",
                "                \"  bind_cols(10^prediction) |>\\n\",\n",
                "                \" ggplot(aes(x = `Time (days)`, y = `CO2 emission (ug/h/g)`)) +\\n\",\n",
                "                \"  geom_point(size = 1.5, alpha = 0.4) +\\n\",\n",
                "                \" facet_grid(`Depth (cm)` ~ `Temperature (°C)`, scales = \\\"free\\\", labeller = labeller(`Depth (cm)` = New.labs_b, `Temperature (°C)` = New.labs)) +\\n\",\n",
                "                \"  geom_line(aes(x = `Time (days)`, y = `.pred`)) +\\n\",\n",
                "                \"scale_y_log10() +\\n\",\n",
                "                \"theme_bw() +\\n\",\n",
                "                \"  xlab(\\\"Incubation time (days)\\\") + ylab(bquote(~CO[2]~ 'emision rate ('*mu~'g'~ h^-1~g^-1*')'))\\n\",\n",
                "                \"  \\n\",\n",
                "                \"pl + theme(axis.text = element_text(face = \\\"bold\\\"),\\n\",\n",
                "                \"           strip.text = element_text(face = \\\"bold\\\"), axis.title.y = element_text(face = \\\"bold\\\"),\\n\",\n",
                "                \"        axis.title=element_text(face = \\\"bold\\\"))\\n\",\n",
                "                \"  \\n\",\n",
                "                \"ggsave(\\\"figures/CO2 emission.png\\\", plot= pl, width = 9, height = 7, dpi = 600)# export plot high resolution\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"## 8. What is the temperature sensitivity across cranberry soil layers?\\n\",\n",
                "                \"\\n\",\n",
                "                \"### 8.1 Fit of Arrhenius equation\\n\",\n",
                "                \"\\n\",\n",
                "                \"The Arrhenius equation has been used to describe temperature sensitivity to CO2 emission. The Arrhenius equation was computed as follows: \\n\",\n",
                "                \"\\n\",\n",
                "                \"\\n\",\n",
                "                \"\\n\",\n",
                "                \"$$k = A e^{{\\\\frac{-Ea}{RT}}}$$\\n\",\n",
                "                \"\\n\",\n",
                "                \"$$log \\\\left( k \\\\right) = log \\\\left( A e^{\\\\frac{-Ea}{RT}} \\\\right)$$\\n\",\n",
                "                \"\\n\",\n",
                "                \"$$log \\\\left( k \\\\right) = log \\\\left( A \\\\right) + log \\\\left(e^{\\\\frac{-Ea}{RT}} \\\\right)$$\\n\",\n",
                "                \"\\n\",\n",
                "                \"$$log \\\\left( k \\\\right) = log \\\\left( A \\\\right) - \\\\frac{1}{T} \\\\times \\\\left(\\\\frac{Ea}{R}\\\\right)$$\\n\",\n",
                "                \"\\n\",\n",
                "                \"Where $A$ is the pre-exponential factor and $Ea$ is activation energy assumed to be independent of temperature, $R$ is the universal gas constant and $T$ is absolute temperature (Kelvin)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"models_co2 <- data_co2 %>%\\n\",\n",
                "                \"  group_by(`Depth (cm)`) %>%\\n\",\n",
                "                \"  summarise(linmod = list(lm(lnK ~ `1/T`)))\\n\",\n",
                "                \"models_co2\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"linmod_coef <- list()\\n\",\n",
                "                \"for (i in seq_along(models_co2$linmod)) linmod_coef[[i]] <- models_co2$linmod[[i]]$coefficients\\n\",\n",
                "                \"linmod_coef <- do.call(rbind.data.frame, linmod_coef)\\n\",\n",
                "                \"names(linmod_coef) <- c(\\\"Intercept\\\", \\\"Slope\\\")\\n\",\n",
                "                \"linmod_coef <- bind_cols(unique(data_co2[\\\"Depth (cm)\\\"]), linmod_coef)\\n\",\n",
                "                \"linmod_coef\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"options(repr.plot.width = 12, repr.plot.height = 6)\\n\",\n",
                "                \"  plot_co2 <- data_co2_clean %>%\\n\",\n",
                "                \"  ggplot(aes(x = `1/T`, y = lnK)) +\\n\",\n",
                "                \"  facet_grid(~`Depth (cm)`, labeller = labeller(`Depth (cm)` = New.labs_b)) +\\n\",\n",
                "                \"  geom_boxplot(aes(group = factor(`1/T`))) +\\n\",\n",
                "                \"stat_regline_equation(aes(label =  paste(..eq.label.., ..rr.label.., sep = \\\"~~\\\")), label.x = 0.00331, label.y = -7) +\\n\",\n",
                "                \"  geom_abline(data = linmod_coef, aes(intercept = Intercept, slope = Slope), lwd = 1) +\\n\",\n",
                "                \"  labs(y = \\\"ln(K)\\\") +\\n\",\n",
                "                \"theme_bw() +\\n\",\n",
                "                \"theme(strip.text = element_text(face = \\\"bold\\\"), axis.text=element_text(face = \\\"bold\\\"),\\n\",\n",
                "                \"        axis.title=element_text(face = \\\"bold\\\")) \\n\",\n",
                "                \"plot_co2\\n\",\n",
                "                \"ggsave(\\\"figures/Arrhénius équation.png\\\", plot = plot_co2, width = 8, height = 4, dpi = 600)# export plot high resolution\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"### 8.2 Activation Energy computation\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"Activation_energy <- tibble(\\n\",\n",
                "                \"    Soil_layers = c(\\\"10\\\", \\\"20\\\", \\\"30\\\"),\\n\",\n",
                "                \"    intercept = NA,\\n\",\n",
                "                \"    slope = NA,\\n\",\n",
                "                \"    adj_r_sq = NA\\n\",\n",
                "                \")\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"lm_arrhenius <- for (i in 1:nrow(Activation_energy)) {\\n\",\n",
                "                \"    \\n\",\n",
                "                \"    lm_Activation_energy <- data_co2_clean %>%\\n\",\n",
                "                \"      filter(`Depth (cm)` == Activation_energy$Soil_layers[i]) %>%\\n\",\n",
                "                \"      lm(lnK ~ `1/T`, data = .)\\n\",\n",
                "                \"    \\n\",\n",
                "                \"    # intercept\\n\",\n",
                "                \"    Activation_energy$intercept[i] <- coef(lm_Activation_energy)[1]\\n\",\n",
                "                \"\\n\",\n",
                "                \"    # Slope\\n\",\n",
                "                \"     Activation_energy$slope[i] <- coef(lm_Activation_energy)[2]\\n\",\n",
                "                \"   \\n\",\n",
                "                \"    # statistics\\n\",\n",
                "                \"    Activation_energy$adj_r_sq[i] <- summary(lm_Activation_energy)$adj.r.squared\\n\",\n",
                "                \"}\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"R = 8.3144621 / 1000 # Gas constant Kj/mol/K \\n\",\n",
                "                \"Activation_energy <-  Activation_energy %>%\\n\",\n",
                "                \"  mutate(Ea = -slope * R) %>%\\n\",\n",
                "                \"  select(Soil_layers, adj_r_sq, Ea)\\n\",\n",
                "                \"Activation_energy\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"### 8.3 Computing K median in order to compute Q10 value accross soil depth\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"K_median <- aggregate(K ~ `Depth (cm)` + `Temperature (°C)`, data = data_co2_clean, FUN = mean)\\n\",\n",
                "                \"K_median_01 <- K_median %>%\\n\",\n",
                "                \"  pivot_wider(names_from = `Temperature (°C)`, values_from = K)\\n\",\n",
                "                \"\\n\",\n",
                "                \"K_median_01$Q_20_10 <- K_median_01$`20` / K_median_01$`10`\\n\",\n",
                "                \"K_median_01$Q_30_20 <- K_median_01$`30` / K_median_01$`20`\\n\",\n",
                "                \"K_median_01\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"K_median_02 <- aggregate(K ~ `Sites` + `Time (days)` + `Depth (cm)` + `Temperature (°C)`, data = data_co2_clean, FUN = median)\\n\",\n",
                "                \"\\n\",\n",
                "                \"K_median_02 <- K_median_02 %>%\\n\",\n",
                "                \"  pivot_wider(names_from = `Temperature (°C)`, values_from = K)\\n\",\n",
                "                \"\\n\",\n",
                "                \"#K_median_02\\n\",\n",
                "                \"K_median_02$Q_20_10 <- K_median_02$`20` / K_median_02$`10`\\n\",\n",
                "                \"K_median_02$Q_30_20 <- K_median_02$`30` / K_median_02$`20`\\n\",\n",
                "                \"K_median_02 <- K_median_02 %>%\\n\",\n",
                "                \"na.omit(K_median_02)\\n\",\n",
                "                \"data_Q10 <- gather(data = K_median_02, key = `Temperature range`, value = Q10, c(`Q_20_10`, `Q_30_20`),\\n\",\n",
                "                \"                    factor_key=TRUE)\\n\",\n",
                "                \"\\n\",\n",
                "                \"mean_sd_Q10 <- ddply(data_Q10, ~ `Depth (cm)`,\\n\",\n",
                "                \"                  summarise, Q10_mean = mean(Q10, na.rm = TRUE), Q10_se = sd(Q10, na.rm = TRUE) / sqrt(dim(data_Q10)[1]))\\n\",\n",
                "                \"mean_sd_Q10\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"data_Q10$Log_Q10 <- log10(data_Q10$Q10)\\n\",\n",
                "                \"\\n\",\n",
                "                \"Q10_lm <- lm(log(Q10) ~ `Depth (cm)`,\\n\",\n",
                "                \"                     data = data_Q10) \\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"summary(Q10_lm)\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"options(repr.plot.width = 8, repr.plot.height = 4)\\n\",\n",
                "                \"data_Q10$`Layers` <- as.character(data_Q10$`Depth (cm)`)\\n\",\n",
                "                \"ggplot(data=data_Q10, aes(x = `Time (days)`, y = `Q10`)) +\\n\",\n",
                "                \"facet_grid(.~`Depth (cm)`, labeller = labeller(`Depth (cm)` = New.labs_b)) +\\n\",\n",
                "                \"geom_smooth(method = \\\"lm\\\", se = TRUE, color = \\\"Black\\\") +\\n\",\n",
                "                \"geom_point(size = 1.5, alpha = 0.5) +\\n\",\n",
                "                \"theme_bw() + \\n\",\n",
                "                \"theme(strip.text = element_text(face = \\\"bold\\\"), axis.text=element_text(face = \\\"bold\\\"),\\n\",\n",
                "                \"        axis.title=element_text(face = \\\"bold\\\")) \\n\",\n",
                "                \"ggsave(\\\"figures/Variation of Q10 across layers.png\\\", width = 8, height = 4, dpi = 600)# export plot high resolution\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"## 9. Soil description\\n\",\n",
                "                \"\\n\",\n",
                "                \"### 9.1 Soil layers properties\\n\",\n",
                "                \"\\n\",\n",
                "                \"Import data\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"data_carbon_credit <- read_csv2('data/data_carbon_credit.csv')\\n\",\n",
                "                \"data_carbon_credit <- data_carbon_credit %>%\\n\",\n",
                "                \"mutate(`C:N ratio` = `Carbone (%)` / `Nitrogen (%)`)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"Some calculations\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"names(data_carbon_credit)\\n\",\n",
                "                \"data_carbon_credit$`Total porosity`\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"mean_sd_CoverN <- ddply(data_carbon_credit, ~ `Layer (cm)`,\\n\",\n",
                "                \"                       summarise, mean_CoverN = mean(`C:N ratio`, na.rm = TRUE), \\n\",\n",
                "                \"                        se_CoverN = sd(`C:N ratio`, na.rm = TRUE)/sqrt(length(!is.na(`C:N ratio`))))\\n\",\n",
                "                \"mean_sd_CoverN\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"plot_desc <- function(y, ylab){\\n\",\n",
                "                \"  New.labs_c <-  c(\\\"Site/A9\\\", \\\"Site/45\\\") # Change labels\\n\",\n",
                "                \"names(New.labs_c) <- c(\\\"Belanger/ A9\\\", \\\"Fortier/ 45\\\")\\n\",\n",
                "                \"  ggplot(data_carbon_credit, aes(`Layer (cm)`, y)) +\\n\",\n",
                "                \"facet_grid( . ~ `Location`, scales = \\\"free\\\", labeller = labeller(`Location` = New.labs_c)) +\\n\",\n",
                "                \"geom_boxplot() + \\n\",\n",
                "                \"theme(strip.text = element_text(size = 11), axis.text=element_text(size=11),\\n\",\n",
                "                \"        axis.title=element_text(size=11)) +\\n\",\n",
                "                \"    labs(y = ylab)\\n\",\n",
                "                \"  }\\n\",\n",
                "                \"\\n\",\n",
                "                \"plot1 <- plot_desc(data_carbon_credit$`C stock (kg m-3)`, \\\"C stock (kg m-3)\\\")\\n\",\n",
                "                \"plot2 <- plot_desc(data_carbon_credit$`C:N ratio`, \\\"C:N ratio\\\")\\n\",\n",
                "                \"plot3 <- plot_desc(data_carbon_credit$`Bulk density (kg m-3)`, \\\"Bulk density (kg m-3)\\\")\\n\",\n",
                "                \"plot4 <- plot_desc(data_carbon_credit$pHCaCl2, \\\"pHCaCl2\\\")\\n\",\n",
                "                \"plot5 <- plot_desc(data_carbon_credit$`Total porosity`, \\\"Total porosity\\\")\\n\",\n",
                "                \"plot6 <- plot_desc(data_carbon_credit$`Water content (%)`, \\\"Water content (%)\\\")\\n\",\n",
                "                \"\\n\",\n",
                "                \"options(repr.plot.width = 8, repr.plot.height = 6)\\n\",\n",
                "                \"figure <- ggarrange(plot1, plot2, plot3, plot4, plot5, plot6,\\n\",\n",
                "                \"                    labels = c(\\\"A\\\", \\\"B\\\", \\\"C\\\", \\\"D\\\", \\\"E\\\", \\\"F\\\"), label.x = 0.05, label.y = 1.01,\\n\",\n",
                "                \"                    ncol = 2, nrow = 3)\\n\",\n",
                "                \"figure\\n\",\n",
                "                \"ggsave(\\\"figures/Soil description.png\\\", width = 8, height = 5, dpi = 600)# export plot high resolution\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"## 9.2 C:N ratio in alternate sublayers of sand and organic matter\\n\",\n",
                "                \"\\n\",\n",
                "                \"Data loading\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"Carbon_credit <- read_csv2('data/data_carbon_sublayer.csv')\\n\",\n",
                "                \"Carbon_credit\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"C:N ratio computation\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"Carbon_credit <- Carbon_credit %>%\\n\",\n",
                "                \"mutate(`C/N` = C_pourc/N_pourc)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": [\n",
                "                \"Generating the plots\\n\",\n",
                "                \"\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"code\",\n",
                "            \"execution_count\": null,\n",
                "            \"metadata\": {},\n",
                "            \"outputs\": [],\n",
                "            \"source\": [\n",
                "                \"options(repr.plot.width=8, repr.plot.height=4)\\n\",\n",
                "                \"pd <- position_dodge(width = 0.2)\\n\",\n",
                "                \"New.labs_d <-  c(\\\"Site/A9\\\", \\\"Site/45\\\") # Change labels\\n\",\n",
                "                \"names(New.labs_d) <- c(\\\"Belanger/A9\\\", \\\"Fortier/45\\\")\\n\",\n",
                "                \"\\n\",\n",
                "                \"p <- ggplot(data=Carbon_credit, aes(x= `Depht (cm)`, y= `C/N`)) +\\n\",\n",
                "                \"facet_grid(.~Site, labeller = labeller(`Site` = New.labs_d)) +\\n\",\n",
                "                \"geom_line(linetype = \\\"twodash\\\") +\\n\",\n",
                "                \"geom_point(aes(shape = `Soil texture`, fill = `Soil texture`), size = 3) +\\n\",\n",
                "                \"scale_shape_manual(values=c(21, 21))+\\n\",\n",
                "                \"scale_fill_manual(values = c(\\\"#000000\\\", \\\"#FFFFFF\\\")) +\\n\",\n",
                "                \"scale_y_continuous(breaks = 5*0:1000,\\n\",\n",
                "                \"                     expand = expand_scale(add = 5)) +\\n\",\n",
                "                \"scale_x_continuous(breaks = 5*0:1000,\\n\",\n",
                "                \"                     expand = expand_scale(add = 5)) +\\n\",\n",
                "                \"theme(strip.text = element_text(face = \\\"bold\\\"), axis.text=element_text(face = \\\"bold\\\"),\\n\",\n",
                "                \"        axis.title=element_text(face = \\\"bold\\\") , legend.title= element_text(face = \\\"bold\\\"),\\n\",\n",
                "                \"     legend.text = element_text(face = \\\"bold\\\")) +\\n\",\n",
                "                \"  labs(y= \\\"C/N Ratio\\\") \\n\",\n",
                "                \"p + coord_flip()\\n\",\n",
                "                \"ggsave(\\\"figures/(C_over_N).png\\\", width = 8, height = 4, dpi = 800)\\n\"\n",
                "            ]\n",
                "        },\n",
                "        {\n",
                "            \"cell_type\": \"markdown\",\n",
                "            \"metadata\": {},\n",
                "            \"source\": \"\\n\"\n",
                "        }\n",
                "    ],\n",
                "    \"metadata\": {\n",
                "        \"anaconda-cloud\": \"\",\n",
                "        \"kernelspec\": {\n",
                "            \"display_name\": \"R\",\n",
                "            \"langauge\": \"R\",\n",
                "            \"name\": \"ir\"\n",
                "        },\n",
                "        \"language_info\": {\n",
                "            \"codemirror_mode\": \"r\",\n",
                "            \"file_extension\": \".r\",\n",
                "            \"mimetype\": \"text/x-r-source\",\n",
                "            \"name\": \"R\",\n",
                "            \"pygments_lexer\": \"r\",\n",
                "            \"version\": \"3.4.1\"\n",
                "        }\n",
                "    },\n",
                "    \"nbformat\": 4,\n",
                "    \"nbformat_minor\": 1\n",
                "}\n"
            ]
        }
    ],
    "metadata": {
        "anaconda-cloud": "",
        "kernelspec": {
            "display_name": "R",
            "langauge": "R",
            "name": "ir"
        },
        "language_info": {
            "codemirror_mode": "r",
            "file_extension": ".r",
            "mimetype": "text/x-r-source",
            "name": "R",
            "pygments_lexer": "r",
            "version": "3.4.1"
        }
    },
    "nbformat": 4,
    "nbformat_minor": 1
}
